# Data Cleaning using Pandas:

This dataset was used as a tool to practice concepts learned related to Python and the Pandas library. The intent was to clean the data from a dataset, downloaded from kaggle, which is composed by the different plant and animal species found in American National Parks.

Started off by importing libraries I could need for the data cleaning and imported the csv file:

In [1]:
import pandas as pd
import numpy as np

In [2]:
species_df = pd.read_csv("species.csv")
species_df.head()

C:\Users\Manel Maria Pinto\AppData\Local\Temp\ipykernel_18820\1586221993.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  species_df = pd.read_csv("raw_species.csv")


,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN


First and foremost I decided to remove possible duplicates, as there shouldn't be any rows with the same information in all the same columns. 

In [3]:
species_df = species_df.drop_duplicates()
species_df.describe()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
count,119248,119248,119248,117776,117736,119248,92101,119242,99106,94203,76306,20157,4718,5
unique,119248,56,14,554,2332,46022,35825,53,7,5,8,24,11,3
top,ACAD-1000,Great Smoky Mountains National Park,Vascular Plant,Poales,Asteraceae,Falco peregrinus,Brachythecium Moss,Approved,Present,Native,Unknown,Breeder,Species of Concern,Threatened
freq,1,6623,65221,11453,8843,56,75,86254,83278,75950,28119,12214,3843,2


Upon running the describe function, its clear that the 13th column is not correctly filled, only 5 filled rows, with 3 unique values and no column name:

In [4]:
unnamed_13 = species_df.iloc[:,13].dropna()
unnamed_13

6441             Endangered
31786            Threatened
31826            Threatened
44733    Species of Concern
44944    Species of Concern
Name: Unnamed: 13, dtype: object

Checking the non null values on the 13th column, it is clear that the values present on this column are supposed to be located on the previous column "Conservation Status".

Checking again the describe function output, 53 unique Record Status seems too big a number of different values to be correct.

In [5]:
species_df["Record Status"] = species_df["Record Status"].str.strip("123._/*")
species_df["Record Status"].iloc[79013] = np.nan
record_status= species_df.iloc[:,7].dropna()
record_status = species_df.iloc[:,7].drop_duplicates()
record_status

0                    Approved
10                  In Review
812             Purple Cockle
3280             Dames Rocket
6441                   Manati
27484      Ranchers' Fireweed
31786     Cocodrilo De Tumbes
31826                 Cabezon
44600                  Wapiti
44610     Short-Tailed Weasel
44640      White-Footed Mouse
44666        Northern Goshawk
44678        Northern Pintail
44705         Common Poorwill
44733             Pigeon Hawk
44744                 Bushtit
44759           American Crow
44818                 Catbird
44859                   Robin
44924          Speckled Trout
44933                    Osha
44944               Wild Iris
44968            Fringed Sage
45018                Rushpink
45060            Ground Daisy
45064                Bluebell
45114                  Cenizo
45115               Goosefoot
45127              Claret Cup
45158       Devil's Shoelaces
45169              Verdolagas
45170                   Chico
45194         Skyrocket Gilia
45210     

In [6]:
#Random row presented in the previous output
species_df.iloc[45473]

Species ID                                               GRSA-1876
Park Name              Great Sand Dunes National Park and Preserve
Category                                            Vascular Plant
Order                                                 Ranunculales
Family                                               Ranunculaceae
Scientific Name                              Clematis hirsutissima
Common Names                                            Sugarbowls
Record Status                                       Leather Flower
Occurrence                                                Approved
Nativeness                                                 Present
Abundance                                                   Native
Seasonality                                                   Rare
Conservation Status                                            NaN
Unnamed: 13                                                    NaN
Name: 45473, dtype: object

Looking at the values obtained, there are common names populating the "Record Status" column, picking one of these rows, its clear that that extra common name is pushing the whole row's data one column to the right, leaving all the information in the wrong place and populating the 13th row with wrong data. It is necessary, in this cases, to clear the information wrongly placed in the "Record Status" column and pull all the data in the subsequent columns, one space to the left.

In order to keep the information in the dataframe in case it is needed, its necessary to join those common names with the ones already corretly placed on the "Common Names" column and delete them from the "Record Status" column afterwards:

In [7]:
condition = ~(species_df['Record Status'].isin(['Approved', 'In Review']))
species_df.loc[condition, 'Common Names'] = species_df["Common Names"] + ", " + species_df["Record Status"]
species_df.loc[condition, 'Record Status'] = ""

record_status = species_df.iloc[:,7].drop_duplicates()
record_status

0       Approved
10     In Review
812             
Name: Record Status, dtype: object

Its clear that the "Record Status" column is only populated by the intended values and a bunch of blank cells.
In the next step I wanted to check every row on these column to see if it was blank and, in those cases, pull all data one column to the left, starting from the "Record Status" column forward:

In [8]:
for index, row in species_df.iterrows():    
    if row['Record Status'] == '':
        species_df.iloc[index, 7:-1] = row[8:].values
        species_df.iloc[index, -1] = ''

In [9]:
species_df.iloc[45158]

Species ID                                               GRSA-1561
Park Name              Great Sand Dunes National Park and Preserve
Category                                            Vascular Plant
Order                                               Caryophyllales
Family                                                Polygonaceae
Scientific Name                                Polygonum aviculare
Common Names                          Knotweed,  Devil's Shoelaces
Record Status                                             Approved
Occurrence                                           Not Confirmed
Nativeness                                                  Native
Abundance                                                      NaN
Seasonality                                                    NaN
Conservation Status                                            NaN
Unnamed: 13                                                       
Name: 45158, dtype: object

By checking again one of the rows that was wrongly filled, the information seems to be in the correct place now.

In [10]:
species_df.describe()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
count,119248,119248,119248,117776,117736,119248,92094,119248,99106,94197,76302,20124,4709,57
unique,119248,56,14,554,2332,46022,35855,2,5,3,6,20,8,1
top,ACAD-1000,Great Smoky Mountains National Park,Vascular Plant,Poales,Asteraceae,Falco peregrinus,Brachythecium Moss,Approved,Present,Native,Unknown,Breeder,Species of Concern,
freq,1,6623,65221,11453,8843,56,75,86305,83331,75990,28121,12221,3845,57


In [11]:
unnamed_13 = species_df.iloc[:,13].drop_duplicates()
unnamed_13

0      NaN
812       
Name: Unnamed: 13, dtype: object

Checking again the describe function's output, is clear the 13th column is now clear of data (only has blank and null values), so it isn't needed anymore:

In [12]:
species_df = species_df.drop(columns = 'Unnamed: 13')

In [13]:
species_df["Order"] = species_df["Order"].str.strip("123._/*")
species_df["Family"] = species_df["Family"].str.strip("123._/*")
species_df["Scientific Name"] = species_df["Scientific Name"].str.strip("123._/*")
species_df["Common Names"] = species_df["Common Names"].str.strip("123._/*")


species_df.head()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN


In [14]:
species_df.to_csv('clean_species.csv', index=False)